In [2]:
#Document qui vise à explorer le dataset de la base de données de KBO

#Importation des librairies
import os
import pandas as pd
import numpy as np
import dask.dataframe as dd
from dask import delayed    
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import joblib

In [3]:
# Spécifier les types de données
dtypes_activity = {
    'EntityNumber': 'object',
    'ActivityGroup': 'int64',
    'NaceVersion': 'int64',
    'NaceCode': 'int64',
    'Classification': 'object'
}

dtypes_address = {
    'EntityNumber': 'object',
    'TypeOfAddress': 'object',
    'CountryNL': 'object',
    'CountryFR': 'object',
    'Zipcode': 'object',  # Spécifier Zipcode comme objet
    'MunicipalityNL': 'object',
    'MunicipalityFR': 'object',
    'StreetNL': 'object',
    'StreetFR': 'object',
    'HouseNumber': 'object',
    'Box': 'object',
    'ExtraAddressInfo': 'object',
    'DateStrikingOff': 'object'
}

dtypes_enterprise = {
    'EnterpriseNumber': 'object',
    'Status': 'object',
    'JuridicalSituation': 'int64',
    'TypeOfEnterprise': 'int64',
    'JuridicalForm': 'float64',  # Peut contenir des NaNs
    'JuridicalFormCAC': 'float64',  # Peut contenir des NaNs
    'StartDate': 'object'
}

In [4]:
# Chemin relatif vers le dossier de données
data_dir = os.path.join('..', '..', 'data')

# Chemins complets vers chaque fichier CSV
activity_path = os.path.join(data_dir, 'activity.csv')
address_path = os.path.join(data_dir, 'address.csv')
enterprise_path = os.path.join(data_dir, 'enterprise.csv')

In [5]:
# Chargement des datasets
activity_df = dd.read_csv(activity_path, dtype=dtypes_activity)
address_df = dd.read_csv(address_path, dtype=dtypes_address)
enterprise_df = dd.read_csv(enterprise_path, dtype=dtypes_enterprise)

def optimize_dataframe(df):
    for col in df.columns:
        col_type = df[col].dtype
        if col_type == 'float64':
            df[col] = df[col].astype('float32')
        elif col_type == 'int64':
            df[col] = df[col].astype('int32')
        elif col_type == 'object':
            num_unique_values = len(df[col].unique())
            num_total_values = len(df[col])
            if num_unique_values / num_total_values < 0.5:
                df[col] = df[col].astype('category')
    return df

# Optimiser les datasets
activity_df = optimize_dataframe(activity_df)
address_df = optimize_dataframe(address_df)
enterprise_df = optimize_dataframe(enterprise_df)

In [6]:
# Fusionner les datasets en utilisant EntityNumber comme clé primaire
merged_df = activity_df.merge(address_df, on='EntityNumber', how='left')
merged_df = merged_df.merge(enterprise_df, left_on='EntityNumber', right_on='EnterpriseNumber', how='left')


In [7]:
# Sélectionner les caractéristiques pertinentes
features = merged_df[['ActivityGroup', 'NaceVersion', 'NaceCode', 'Zipcode', 'JuridicalSituation', 'TypeOfEnterprise']]
target = merged_df['Classification'].apply(lambda x: 1 if x == 'MAIN' else 0, meta=('x', 'int64'))  # Convertir MAIN à 1 et SECONDARY à 0


In [10]:
# Remplacer les valeurs manquantes pour les colonnes numériques et non numériques
def fill_missing_values(df):
    for col in df.columns:
        if df[col].dtype in ['int64', 'float64']:
            df[col] = df[col].fillna(df[col].mean())
        else:
            df[col] = df[col].fillna(df[col].mode()[0])
    return df

# Appliquer la fonction aux partitions
features = features.map_partitions(fill_missing_values)

In [12]:
# Convertir les DataFrames Dask en DataFrames pandas par partitions
def compute_partition(part):
    return part.compute()

# Initialiser des listes pour stocker les partitions pandas
X_list = []
y_list = []

# Traiter les partitions une par une
for part in features.to_delayed():
    X_list.append(compute_partition(part))

for part in target.to_delayed():
    y_list.append(compute_partition(part))

# Concaténer les partitions en un seul DataFrame pandas
X = pd.concat(X_list, ignore_index=True)
y = pd.concat(y_list, ignore_index=True)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraîner un modèle de classification (Random Forest)
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Prédire sur l'ensemble de test
y_pred = model.predict(X_test)

# Évaluer le modèle
print(classification_report(y_test, y_pred))

MemoryError: 

In [ ]:
joblib.dump(model, 'model.pkl')  # Sauvegarder le modèle